In [ ]:
# This code is based from the examples given in the detectron2 documentation.
#
# Authors:  Yuxin Wu, Alexander Kirillov, Francisco Massa,
#           Wan-Yen Lo and Ross Girshick
# Company:  Facebook AI Research
# Year:     2019
# Title:    plain_train.py
# Type:     Source Code
# URL:      https://github.com/facebookresearch/detectron2/blob/master/tools/plain_train_net.py
#
#
# Authors:  Yuxin Wu, Alexander Kirillov, Francisco Massa,
#           Wan-Yen Lo and Ross Girshick
# Company:  Facebook AI Research
# Year:     2019
# Title:    Detectron2 Tutorial.ipynb
# Type:     Source Code
# URL:      https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5




Show information about the assigned GPU resource:

In [ ]:
!nvidia-smi

Setup and instal dependencies:

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision

print(torch.__version__, torch.cuda.is_available())
!gcc --version

In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

In [ ]:
!pip install wandb -q

In [ ]:
import wandb
import os

In [ ]:
import detectron2

# Import common libraries
import os
import json
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# Import detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer  # Look into moving to a custom training loop to add weights & biases
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog
from detectron2.structures import BoxMode
from detectron2.config import get_cfg

Connect to Google Drive (dataset stored on G drive due to size):

In [ ]:
%cd /content/drive/My\ Drive/Project/

Define program setup and training parameters:

In [ ]:
os.environ['WANDB_API_KEY'] = 'your_weights_and_biases_API_key'

# TOGGLE PROGRAM FUNCTIONALITY
TEST_INPUT = False  # Toggles image read test
TRAINING_CURVES = False  # Toggles Tensorboard training curves


# PROGRAM CONSTANTS
ROOT_DIR = "10_class"
WANDB_PROJ = 'multi-class'
model_name = "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"

# TRAINING PARAMETERS
num_classes = 10
workers = 2
ims_per_batch = 4
seed = 27

lr = 0.0025
warmup_iters = 500
max_iter = 25000
step_low = 15000
step_high = 25000
gamma = 0.2
momentum = 0.90
eval_period = 2500

# FILE PATHS
TRAIN_DATASET_NAME = ROOT_DIR + "_train"
TRAIN_ANNOTATIONS = ROOT_DIR + "/train/annotations.json"
TRAIN_DIR = ROOT_DIR + "/train"

TEST_DATASET_NAME = ROOT_DIR + "_val"
TEST_ANNOTATIONS = ROOT_DIR + "/val/annotations.json"
TEST_DIR = ROOT_DIR + "/val"

output = ROOT_DIR + '/output'

Register the dataset and add it the dataset catalog:

In [ ]:
try:
  register_coco_instances(TRAIN_DATASET_NAME, {}, TRAIN_ANNOTATIONS, TRAIN_DIR)
  register_coco_instances(TEST_DATASET_NAME, {}, TEST_ANNOTATIONS, TEST_DIR)
except (AssertionError):
  print('Dataset has aready been registered!')

train_metadata = MetadataCatalog.get(TRAIN_DATASET_NAME)
dataset_dicts = DatasetCatalog.get(TRAIN_DATASET_NAME)

Get a dictionary containing the number of object instances per class
in the dataset. This will be logged to Weights and Biases later. 

In [ ]:
hist_bins = np.arange(num_classes + 1)
histogram = np.zeros((num_classes,), dtype=np.int)
for entry in dataset_dicts:
  annos = entry["annotations"]
  classes = [x["category_id"] for x in annos if not x.get("iscrowd", 0)]
  histogram += np.histogram(classes, bins=hist_bins)[0]

class_instances = {('class_instances.' + train_metadata.thing_classes[i]): int(count) for i, count in enumerate(histogram)}

Show a sample of image inputs for debug purposes:

In [ ]:
if TEST_INPUT:
  for d in random.sample(dataset_dicts, 3):
      img = cv2.imread(d["file_name"])
      visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
      out = visualizer.draw_dataset_dict(d)
      cv2_imshow(out.get_image()[:, :, ::-1])

Detectron2 training & test functions: 

In [ ]:
import logging
import os
from shutil import copyfile
from collections import OrderedDict
import torch
from torch.nn.parallel import DistributedDataParallel

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2.config import get_cfg
from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.engine import default_argument_parser, default_setup, launch
from detectron2.evaluation import (
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    SemSegEvaluator,
    inference_on_dataset,
    print_csv_format,
)
from detectron2.modeling import build_model
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.utils.events import (
    CommonMetricPrinter,
    EventStorage,
    JSONWriter,
    TensorboardXWriter,
)

# Setup logger
logger = logging.getLogger("detectron2")

In [ ]:
def get_evaluator(cfg, dataset_name, output_folder=None):
    """
    Create evaluator(s) for a given dataset.
    This uses the special metadata "evaluator_type" associated with each builtin dataset.
    For your own dataset, you can simply create an evaluator manually in your
    script and do not have to worry about the hacky if-else logic here.

    # TODO: Edit docstring

    """
    if output_folder is None:
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
    evaluator_list = []

    evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type
    if evaluator_type in ["sem_seg", "coco_panoptic_seg"]:
        evaluator_list.append(
            SemSegEvaluator(
                dataset_name,
                distributed=True,
                num_classes=cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES,
                ignore_label=cfg.MODEL.SEM_SEG_HEAD.IGNORE_VALUE,
                output_dir=output_folder,
            )
        )
    if evaluator_type in ["coco", "coco_panoptic_seg"]:
        evaluator_list.append(COCOEvaluator(dataset_name, cfg, True, output_folder))
    if evaluator_type == "coco_panoptic_seg":
        evaluator_list.append(COCOPanopticEvaluator(dataset_name, output_folder))

    if len(evaluator_list) == 0:
        raise NotImplementedError(
            "no Evaluator for the dataset {} with the type {}".format(dataset_name, evaluator_type)
        )
    if len(evaluator_list) == 1:
        return evaluator_list[0]
    return DatasetEvaluators(evaluator_list)

In [ ]:
def do_test(cfg, model):
    """

    # TODO: Write docstring
    """
    # Initialise results dictionary
    results = OrderedDict()

    # Loop through the datasets in the config file
    for dataset_name in cfg.DATASETS.TEST:
        data_loader = build_detection_test_loader(cfg, dataset_name)

        # Generate the evaluator
        evaluator = get_evaluator(
            cfg,
            dataset_name,
            os.path.join(cfg.OUTPUT_DIR, "inference", dataset_name)
        )

        # Run inference and add to results dictionary
        results_i = inference_on_dataset(model, data_loader, evaluator)
        results[dataset_name] = results_i

        # Log the result set to weights and biases
        result_log = {}
        result_dict = results_i
        for iou_type in result_dict:
          for metric, result in result_dict[iou_type].items():
            metric_log = f"{iou_type}_{metric}"
            result_log[metric_log] = result

        logger.debug('Log the eval results on Weights & Biases')
        wandb.log(result_log)

        # Print to terminal
        if comm.is_main_process():
            logger.info("Evaluation results for {} in csv format:".format(dataset_name))
            print_csv_format(results_i)
    if len(results) == 1:
        results = list(results.values())[0]

    return results

In [ ]:
def do_train(cfg, model, resume=False):
    """

    # TODO: Write docstring
    """
    # Set the model to train
    model.train()

    # Create torch optimiser & schedulars
    optimizer = build_optimizer(cfg, model)
    scheduler = build_lr_scheduler(cfg, optimizer)

    # Create a torch checkpointer
    checkpointer = DetectionCheckpointer(
        model, cfg.OUTPUT_DIR, optimizer=optimizer, scheduler=scheduler
    )

    # Create starting checkpoint i.e. pre-trained model using weights from config
    start_iter = (
        checkpointer.resume_or_load(cfg.MODEL.WEIGHTS, resume=resume).get("iteration", -1) + 1
    )

    # Define the number of iterations
    max_iter = cfg.SOLVER.MAX_ITER

    # Create a periodic checkpointer at the configured period
    periodic_checkpointer = PeriodicCheckpointer(
        checkpointer, cfg.SOLVER.CHECKPOINT_PERIOD, max_iter=max_iter
    )

    # Export checkpoint data to terminal, JSON & tensorboard files
    writers = (
        [
            CommonMetricPrinter(max_iter),
            JSONWriter(os.path.join(cfg.OUTPUT_DIR, "metrics.json")),
            TensorboardXWriter(cfg.OUTPUT_DIR),
        ]
        if comm.is_main_process()
        else []
    )

    # Create a data loader to supply the model with training data
    data_loader = build_detection_train_loader(cfg)

    logger.info("Starting training from iteration {}".format(start_iter))
    with EventStorage(start_iter) as storage:
        for data, iteration in zip(data_loader, range(start_iter, max_iter)):
            iteration = iteration + 1
            storage.step()

            loss_dict = model(data)
            losses = sum(loss_dict.values())
            assert torch.isfinite(losses).all(), loss_dict

            loss_dict_reduced = {k: v.item() for k, v in comm.reduce_dict(loss_dict).items()}
            losses_reduced = sum(loss for loss in loss_dict_reduced.values())
            if comm.is_main_process():
                storage.put_scalars(total_loss=losses_reduced, **loss_dict_reduced)
          
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()
            storage.put_scalar("lr", optimizer.param_groups[0]["lr"], smoothing_hint=False)
            scheduler.step()

            # If eval period has been set, run test at defined interval
            if (
                cfg.TEST.EVAL_PERIOD > 0
                and iteration % cfg.TEST.EVAL_PERIOD == 0
                and iteration != max_iter
            ):
                do_test(cfg, model)
                comm.synchronize()

            if iteration - start_iter > 5 and (iteration % 20 == 0 or iteration == max_iter):
                logger.debug('Logging iteration and loss to Weights & Biases')
                wandb.log({"iteration": iteration})
                wandb.log({"total_loss": losses_reduced})
                wandb.log(loss_dict_reduced)

                for writer in writers:
                    writer.write()
            periodic_checkpointer.step(iteration)

In [ ]:
def setup(args, model_name):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)

    cfg.freeze()

    # Log the configuration file to OUTPUT_DIR
    default_setup(
        cfg, args
    )

    # Set up the weights and biases project
    logger.debug('Initialising Weights & Biases project')
    wandb.init(project=WANDB_PROJ, sync_tensorboard=False)

    # Load the yaml file and export it to wandb
    cfg_export = cfg.load_yaml_with_base(os.path.join(cfg.OUTPUT_DIR, "config.yaml"))
    logger.debug("Saving cfg file to Weights & Biases")
    wandb.config.update(cfg_export)

    # Log train and val set sizes
    wandb.config.update({"CONFIG_FILE": model_name})
    wandb.config.update({"train_imgs": len(dataset_dicts)})
    wandb.config.update({"test_imgs": len(DatasetCatalog.get(TEST_DATASET_NAME))})
    wandb.config.update(class_instances)

    return cfg

In [ ]:
def main(args, model_name):
    # Initialise the configuration datastructure
    cfg = setup(args, model_name)

    # Build a model from the configuration file
    model = build_model(cfg)

    logger.info("Model:\n{}".format(model))

    # If the 'eval_only' argument is present, load the last checkpoint
    # and return the results of the test function
    if args.eval_only:
        DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            cfg.MODEL.WEIGHTS, resume=args.resume
        )
        return do_test(cfg, model)

    # Run the training loop
    do_train(cfg, model, resume=args.resume)

    # Save model to weights and biases
    logger.debug('Saving model to Weights & Biases')
    copyfile(ROOT_DIR + '/output/model_final.pth', wandb.run.dir + '/model_final.pth')

    # Return the results of the model test function
    return do_test(cfg, model)

In [ ]:
# Set up an argument string to pass into the main function

config = model_zoo.get_config_file(model_name)
weights = model_zoo.get_checkpoint_url(model_name)

arg_string = f"--config-file {config} \
              MODEL.WEIGHTS {weights} \
              OUTPUT_DIR {output} \
              DATASETS.TRAIN ('{TRAIN_DATASET_NAME}',) \
              DATASETS.TEST ('{TEST_DATASET_NAME}',) \
              DATALOADER.NUM_WORKERS {workers} \
              SOLVER.IMS_PER_BATCH {ims_per_batch} \
              SOLVER.BASE_LR {lr} \
              SOLVER.WARMUP_ITERS {warmup_iters} \
              SOLVER.MAX_ITER {max_iter} \
              SOLVER.STEPS ({step_low},{step_high}) \
              SOLVER.GAMMA {gamma} \
              SOLVER.MOMENTUM {momentum} \
              SEED {seed} \
              MODEL.ROI_HEADS.NUM_CLASSES {num_classes} \
              MODEL.RETINANET.NUM_CLASSES {num_classes} \
              TEST.EVAL_PERIOD {eval_period}".split()

parser = default_argument_parser()
args = parser.parse_args(arg_string)

main(args, model_name)